In [1]:
from transformers import ViTImageProcessor, ViTForImageClassification, ViTModel, ViTConfig
from maskgen.utils import get_preprocess, collate_fn, load_imagenet
# from maskgen.utils.img_utils import plot_overlap_np
from torch.utils.data import DataLoader
import torch
import json
import os
from tqdm import tqdm
import numpy as np
from typing import Dict, Any

config = {
        "pretrained_name": "google/vit-base-patch16-224",
        "results_path": "/scratch365/dpan/new_results/gradcam",
        "max_samples": 100,
        "dataset_split": "tiny",
        "num_samples": 1000,
        "batch_size":1,
}

In [2]:
from attvis.baselines.ViT.ViT_new import vit_base_patch16_224 as vit_LRP
from attvis.baselines.ViT.ViT_explanation_generator import Baselines
from attvis.samples.CLS2IDX import CLS2IDX

# initialize ViT pretrained
model = vit_LRP(pretrained=True).cuda()
model.eval()
attribution_generator = Baselines(model)



ModuleNotFoundError: No module named 'baselines'

In [4]:
from maskgen.baselines.gradshap import GradShapAnalyzer, downsample_attribution

image = get_image_example(2)
inputs = processor(images=image, return_tensors="pt").to(device)
pixel_values = inputs.pixel_values.to(device)

# Initialize analyzer with wrapped model
analyzer = GradShapAnalyzer(
    model=target_model,
    device=device
)

# Get attribution for single image
attribution_map, pred_class, confidence = analyzer.get_attribution(pixel_values)

print(f"Predicted class: {pred_class}")
print(f"Confidence: {confidence:.3f}")

Predicted class: 284
Confidence: 0.964


In [5]:
from maskgen.utils.img_utils import plot_overlap_np
from maskgen.utils.data_utils import get_imagenet_dataloader

# get dataloader
dataloader = get_imagenet_dataloader(split='tiny', 
                                    batch_size=config['batch_size'], 
                                    processor=processor, 
                                    shuffle=False,
                                    num_samples=config['num_samples'])

Repo card metadata block was not found. Setting CardData to empty.


In [6]:
all_inputs = []
all_heatmaps = []

for i, batch in tqdm(enumerate(dataloader), total=len(dataloader), desc="Processing batches"):
    pixel_values = batch['pixel_values'].to(device)
    labels = batch['labels'].to(device)
    attribution_map, pred_class, confidence = analyzer.get_attribution(pixel_values)
    attribution_map = downsample_attribution(attribution_map, patch_size=16)
    attribution_map = np.expand_dims(attribution_map, axis=0)

    inputs_np = pixel_values.cpu().numpy()
    heatmap_np = attribution_map
    all_inputs.append(inputs_np)
    all_heatmaps.append(heatmap_np)

all_inputs = np.concatenate(all_inputs, axis=0)
all_heatmaps = np.concatenate(all_heatmaps, axis=0)
    

Processing batches: 100%|██████████| 1000/1000 [03:31<00:00,  4.73it/s]


In [7]:
from maskgen.utils.save_utils import save_pixel_heatmap_pairs

save_path = config['results_path']
if not os.path.exists(save_path):
    os.makedirs(save_path)
save_path = os.path.join(save_path, 'pixel_heatmap_pairs.npz')
save_pixel_heatmap_pairs(all_inputs, all_heatmaps, save_path)